In [1]:
import sys, os
if os.path.abspath(os.pardir) not in sys.path:
    sys.path.insert(0, os.path.abspath(os.pardir))
import CONFIG

%reload_ext autoreload
%autoreload 2

In [2]:
sys.path.insert(0, os.path.join(CONFIG.CFG.BASEPATH, "Yet-Another-EfficientDet-Pytorch"))

In [3]:
import subprocess
subprocess.call("git clone https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch", shell=True, cwd=CONFIG.CFG.BASEPATH)

0

In [11]:
import torch
import numpy as np 
import pandas as pd
from backbone import EfficientDetBackbone
from utils.utils import preprocess

In [5]:
anchor_ratios = [(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)]
anchor_scales = [2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]

In [10]:
%%capture
model = EfficientDetBackbone(compound_coef=4, num_classes=1, ratios=anchor_ratios, scales=anchor_scales)
model.load_state_dict(torch.load("efficientdet-d4_14_20235.pth"))
model.requires_grad_(False)
model.eval()

In [15]:
IMG_PATH = os.path.join(CONFIG.CFG.DATA.BASE, "test")

In [14]:
ori_imgs, framed_imgs, framed_metas = preprocess(IMG_PATH, max_size=1024)

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'